In [163]:
#%% Set the directory
path = "/Users/Yannick/Documents/Thesis 2024"
os.chdir(path)


#%%Load data

#Trade datasets
cargo_1 = pd.read_excel("Data/trade_data/cargos (1).xls", "cargo", header=None)
cargo_2 = pd.read_excel("Data/trade_data/cargos (2).xls", "cargo", header=None)
cargo_3 = pd.read_excel("Data/trade_data/cargos (3).xls", "cargo", header=None)

#weight conversions
weight_conversion = pd.read_csv("Data/conversion_weights1.csv")

#%% clean the data
#Concat into single df
trade_data = pd.concat([cargo_1, cargo_2, cargo_3])

#Select first 8 columns
trade_data = trade_data.iloc[:,0:8]

#Name these columns
trade_data.columns = ["book_year", "quantity", "unit", "product", "departure_place", "departure_region", "arrival_place", "arrival_region"]

#Convert - to NAN
trade_data.replace('-', None, inplace=True)

#%%Extract a single year so it can be ordered
trade_data["year"] = trade_data["book_year"].str[:4].astype(int)
trade_data = trade_data.sort_values(by='year')

#%% Convert the quantities to the english format and convert to numeric
trade_data["quantity"] = trade_data["quantity"].str.replace('.', '') #Remove the thousand separators
trade_data["quantity"] = trade_data["quantity"].str.replace(',', '.') #Replace the , with a .
trade_data["quantity"] = pd.to_numeric(trade_data["quantity"]) #Make numeric

#%%Convert similar products to the same sort

#manual input of the different names for commodities based on VOC glossarium
Kaneel = ["kaneel", "boskaneel", "kaneel de matte", "candela"]
Zijde = ["zijde", "ablako", "agoni", "allegias", "ardasse", "ardassina", "armozijn", "baa", "bariga", "bogy", "cabessa", "cannagy", "capitoen", "casagy", "chauls", "coetchiaal", "cora", "dom", "floretzijde", "floszijde", "fora", "gangali", "gasen", "gert-kerckerie", "hittou", "kannegie", "kedgoda pessend", "ketsier", "khan baffy", "legia", "matiaal", "moghta", "mongo", "pangia", "parelzijde", "patteni", "pee", "peling", "poilzijde", "poolzijde", "poilzijde", "potti", "quetchoda-passant", "selvatica", "serge", "sjesum", "tabijnen", "taffachelas", "tafta", "tamut", "tanna-banna", "tanny", "tantianozijde", "thujas", "vloszijde", "volte corte", "zeem"]
Peper = ["peper", "staartpeper", "cubebe", "piper cubebe", "ruagiepeper"]
Nootmuskaat = ["nootmuskaat", "muskaatnoten", "brouwersnoten", "foelienoten", "foelie", "rompen", ""]
Kruidnagel = ["antoffel", "blom", "caplet", "garioffelnagel", "nagel", "kruidnagel"]
Thee = ["thee", "bing", "boei", "congo", "joosjesthee", "pecco", "songlo", "souchong", "tscha"]
Koffie = ["koffie", "bun", "cauwa", "kauwa", "kitscha"]
Suiker = ["suiker", "gula", "jagersuycker", "jagru", "poeyer", "stoksuiker", "kandijsuiker", "tamarinde"]
Opium = ["amfioen", "opium"]

#initiate product category
trade_data["product_category"] = None

#Set the product category to the overarching name based on previous lists
trade_data.loc[trade_data["product"].isin(Kaneel), "product_category"] = "Cinnamon"
trade_data.loc[trade_data["product"].isin(Zijde), "product_category"] = "Silk"
trade_data.loc[trade_data["product"].isin(Peper), "product_category"] = "Pepper"
trade_data.loc[trade_data["product"].isin(Nootmuskaat), "product_category"] = "Nutmeg"
trade_data.loc[trade_data["product"].isin(Kruidnagel), "product_category"] = "Cloves"
trade_data.loc[trade_data["product"].isin(Thee), "product_category"] = "Tea"
trade_data.loc[trade_data["product"].isin(Koffie), "product_category"] = "Coffee"
trade_data.loc[trade_data["product"].isin(Suiker), "product_category"] = "Sugar"
trade_data.loc[trade_data["product"].isin(Opium), "product_category"] = "Opium"

    
#%% Filter for the products
commodities = ["Silk", "Pepper", "Cinnamon", "Nutmeg", "Cloves", "Tea", "Coffee", "Sugar", "Opium"]

conversion_commodities = pd.DataFrame()

for commodity in commodities:
    filtered_trade_data = trade_data[(trade_data["product_category"] == commodity)]
    filtered_weight_conversions = weight_conversion[weight_conversion["commodity"] == commodity]
    merged_data = pd.merge(filtered_trade_data, filtered_weight_conversions)
    conversion_commodities = pd.concat([conversion_commodities, merged_data])


conversion_commodities["weight_tonnes"] = conversion_commodities["quantity"] * conversion_commodities["conversion"] /1000
cleaned_trade_data = conversion_commodities[["year", "weight_tonnes", "product_category", "departure_place", "departure_region", "arrival_place", "arrival_region"]]

cleaned_trade_data = cleaned_trade_data.sort_values(by='year')
cleaned_trade_data = cleaned_trade_data.reset_index(drop=True)

print(conversion_commodities)
print(cleaned_trade_data)

     book_year    quantity unit product departure_place departure_region  \
0    1700/1701   18357.500   lb   zijde         Batavia          Batavia   
1    1700/1701   14287.000   lb   zijde         Batavia          Batavia   
2    1700/1701   16974.375   lb   zijde         Batavia          Batavia   
3    1700/1701   21831.000   lb   zijde         Batavia          Batavia   
4    1700/1701   11618.500   lb   zijde         Batavia          Batavia   
..         ...         ...  ...     ...             ...              ...   
98   1779/1780   12300.000   lb   opium            None         Bengalen   
99   1780/1781  122850.000   lb   opium            None         Bengalen   
100  1780/1781     600.000   lb   opium         Batavia          Batavia   
101  1784/1785   61064.000   lb   opium            None         Bengalen   
102  1786/1787   52950.000   lb   opium            None         Bengalen   

    arrival_place arrival_region  year product_category  conversion commodity  \
0     

In [164]:
# Step 2. Specify a column for the flow volume value
value_suffix = "tonne"

  # Specify (if any) a suffix for the value

# Step 4. (Optional) Customize layout, font, and colors
fontsize = 14  # Set font size of labels
fontfamily = "Helvetica"  # Set font family of plot's text
bgcolor = "SeaShell"  # Set the plot's background color (use color name or hex code)
link_opacity = 0.3  # Set a value from 0 to 1: the lower, the more transparent the links
node_colors = px.colors.qualitative.G10  # Define a list of hex color codes for nodes


cols = ['departure_region', 'arrival_region']  # Define the columns you want to use
weight = "weight_tonnes"

available_years = cleaned_trade_data['year'].unique()
slider_marks = {str(year): '' for year in available_years}

app = Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        options=[
                {"label": "Tea", "value": "Tea"},
                {"label": "Silk", "value": "Silk"},
                {"label": "Cloves", "value": "Cloves"},
                {"label": "Nutmeg", "value": "Nutmeg"},
                {"label": "Pepper", "value": "Pepper"},
                {"label": "Cinnamon", "value": "Cinnamon"},
                {"label": "Sugar", "value": "Sugar"},
                {"label": "Coffee", "value": "Coffee"},
                {"label": "Opium", "value": "Opium"}
            ],
            value='Pepper',
            id='dropdown-selection'
    ),
    
    html.Br(),
    
    dcc.Graph(id='graph-content'),
    html.Br(),
    html.Br(),
    dcc.Slider(
        id='year-slider',
        min=min(available_years),
        max=max(available_years),
        step=1,
        value=min(available_years),  # Set initial value to the minimum year
        marks=slider_marks,
        tooltip={
        "always_visible": True,
        "style": {"color": "LightSteelBlue", "fontSize": "20px"},
    }
)
])

@app.callback(
    [Output('graph-content', 'figure'),
     Output('year-slider', 'marks')],
    [Input('dropdown-selection', 'value'),
     Input('year-slider', 'value')]
)

def update_graph(selected_product, selected_year):
    df_allyears = cleaned_trade_data[cleaned_trade_data["product_category"]==selected_product]
       
    df = df_allyears[df_allyears["year"] == selected_year]

    # Get unique years for the selected commodity
    available_years = df_allyears['year'].unique()

    # Generate marks for the slider based on available years
    slider_marks = {str(year): '' for year in available_years}
    
    if df.empty:
        fig = go.Figure()
        fig.update_layout(title='No data')
        
        # If DataFrame is empty, return a warning message or placeholder figure
        
        return fig, slider_marks
    
    
    else:
        
        s = []  # This will hold the source nodes
        t = []  # This will hold the target nodes
        v = []  # This will hold the flow volumes between the source and target nodes
        labels = np.unique(df[cols].values)  # Collect all the node labels

        # Get all the links between two nodes in the data and their corresponding values
        for c in range(len(cols) - 1):
            s.extend(df[cols[c]].tolist())
            t.extend(df[cols[c + 1]].tolist())
            v.extend(df[weight].tolist())
        links = pd.DataFrame({"source": s, "target": t, "weight": v})  
        links = links.groupby(["source", "target"], as_index=False).agg({"weight": "sum"})

        # Convert list of colors to RGB format to override default gray link colors
        colors = [matplotlib.colors.to_rgb(i) for i in node_colors]  

        # Create objects to hold node/label and link colors
        label_colors, links["link_c"] = [], 0

        # Loop through all the labels to specify color and to use label indices
        c, max_colors = 0, len(colors)  # To loop through the colors array
        for l in range(len(labels)):
            label_colors.append(colors[c])
            link_color = colors[c] + (link_opacity,)  # Make link more transparent than the node
            links.loc[links.source == labels[l], ["link_c"]] = "rgba" + str(link_color)
            links = links.replace({labels[l]: l})  # Replace node labels with the label's index
            if c == max_colors - 1:
                c = 0
            else:
                c += 1

        # Convert colors into RGB string format for Plotly
        label_colors = ["rgb" + str(i) for i in label_colors]

        # Define a Plotly Sankey diagram
        fig = go.Figure( 
            data=[
                go.Sankey(
                    valuesuffix=value_suffix,
                    node=dict(label=labels, color=label_colors),
                    link=dict(
                        source=links["source"],
                        target=links["target"],
                        value=links["weight"],
                        color=links["link_c"],
                    ),
                )
            ]
        )

        # Customize plot based on earlier values
        fig.update_layout(
            title_text = f"Sankey diagram of trade flows in the VOC for {selected_product} in {selected_year}",
            font_size=fontsize,
            font_family=fontfamily,
            width=1200,
            height=600,
            paper_bgcolor=bgcolor,
            title={"y": 0.9, "x": 0.5, "xanchor": "center", "yanchor": "top"},  # Centers title
        )

        return fig, slider_marks

if __name__ == '__main__':
    app.run(jupyter_mode="external", port=8090)

Dash app running on http://127.0.0.1:8090/


/var/folders/g8/0gz518ls57g4pghsnqh995yc0000gn/T/ipykernel_72484/2537216949.py:111: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'rgba(0.2, 0.4, 0.8, 0.3)' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/g8/0gz518ls57g4pghsnqh995yc0000gn/T/ipykernel_72484/2537216949.py:112: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/g8/0gz518ls57g4pghsnqh995yc0000gn/T/ipykernel_72484/2537216949.py:112: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd

/var/folders/g8/0gz518ls57g4pghsnqh995yc0000gn/T/ipykernel_72484/2537216949.py:111: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'rgba(0.2, 0.4, 0.8, 0.3)' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/g8/0gz518ls57g4pghsnqh995yc0000gn/T/ipykernel_72484/2537216949.py:112: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/g8/0gz518ls57g4pghsnqh995yc0000gn/T/ipykernel_72484/2537216949.py:111: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'rgba(0.8627450980392157, 0.2235294117647059, 0.07058823529411765, 0.3)' has dtype incompatible w

/var/folders/g8/0gz518ls57g4pghsnqh995yc0000gn/T/ipykernel_72484/2537216949.py:111: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'rgba(0.2, 0.4, 0.8, 0.3)' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/g8/0gz518ls57g4pghsnqh995yc0000gn/T/ipykernel_72484/2537216949.py:112: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/var/folders/g8/0gz518ls57g4pghsnqh995yc0000gn/T/ipykernel_72484/2537216949.py:111: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'rgba(0.2, 0.4, 0.8, 0.3)' has dtype incompatible with int64, please explicitly cast to a compati